In [ ]:
import os
import json
import time
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import cv2
from sklearn.metrics import accuracy_score
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
USE_CUDA = True
CUDA = USE_CUDA and torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")

if CUDA:
    print('run on %s' % device)

run on cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -avr 'drive/My Drive/ProgettoCV204898/dataset.zip' ./
!cp -f 'drive/My Drive/ProgettoCV204898/testset_keys_1lug2020.txt' ./
!unzip -qq './dataset.zip' -d './'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'drive/My Drive/ProgettoCV204898/dataset.zip' -> './dataset.zip'
replace ./dataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
PATH='./'
DATASET_DIR = './dataset/'
DATASET_FILES = './dataset/examples/'
KEYSET_FILE = './testset_keys_1lug2020.txt'

LABELS = {0 : "block", 1 : "pass", 2 : "run", 3: "dribble",4: "shoot",
          5 : "ball in hand", 6 : "defense", 7: "pick" , 8 : "no_action" , 
          9: "walk" ,10: "discard"}


with open(os.path.join(DATASET_DIR, 'annotation_dict.json')) as fp:
    annotations = json.load(fp)

with open(KEYSET_FILE) as fp:
    keys_test = json.load(fp)

annotationTrain = dict(filter(lambda x: x[0] not in keys_test, annotations.items()))
annotationTest = dict(filter(lambda x: x[0] in keys_test, annotations.items()))

len(annotations), len(annotationTrain), len(annotationTest)


(37085, 25959, 11126)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(list(annotationTrain.keys()), list(annotationTrain.values()), test_size=0.01, random_state=42)
X_test,y_test=list(annotationTest.keys()),list(annotationTest.values())


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
 
class S3D(nn.Module):
    def __init__(self, num_class):
        super(S3D, self).__init__()
        self.base = nn.Sequential(
            SepConv3d(3, 64, kernel_size=7, stride=2, padding=3),                           #Sep-Conv7x7x7
            nn.MaxPool3d(kernel_size=(1,3,3), stride=(1,2,2), padding=(0,1,1)),             #1x3x3 MaxPool
            BasicConv3d(64, 64, kernel_size=1, stride=1),                                   #1x1x1 Conv
            SepConv3d(64, 192, kernel_size=3, stride=1, padding=1),                         #Sep-Conv 3x3x3 (Conv3D 1x3x3->Conv3D 3x1x1)
            nn.MaxPool3d(kernel_size=(1,3,3), stride=(1,2,2), padding=(0,1,1)),             #1x3x3 MaxPool
            Mixed_3b(),
            Mixed_3c(),
            nn.MaxPool3d(kernel_size=(3,3,3), stride=(2,2,2), padding=(1,1,1)),
            Mixed_4b(),
            Mixed_4c(),
            Mixed_4d(),
            Mixed_4e(),
            Mixed_4f(),
            nn.MaxPool3d(kernel_size=(2,2,2), stride=(2,2,2), padding=(0,0,0)),
            Mixed_5b(),
            Mixed_5c(),
        )
        self.fc = nn.Sequential(nn.Conv3d(1024, num_class, kernel_size=1, stride=1, bias=True),)
 
    def forward(self, x):
        y = self.base(x)
        y = F.avg_pool3d(y, (2, y.size(3), y.size(4)), stride=1)
        y = self.fc(y)
        y = y.view(y.size(0), y.size(1), y.size(2))
        logits = torch.mean(y, 2)
 
        return logits

In [ ]:
class BasicConv3d(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride, padding=0):
        super(BasicConv3d, self).__init__()
        self.conv = nn.Conv3d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm3d(out_planes, eps=1e-3, momentum=0.001, affine=True)
        self.relu = nn.ReLU()
 
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

In [ ]:
class SepConv3d(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride, padding=0):
        super(SepConv3d, self).__init__()
        self.conv_s = nn.Conv3d(in_planes, out_planes, kernel_size=(1,kernel_size,kernel_size), stride=(1,stride,stride), padding=(0,padding,padding), bias=False)
        self.bn_s = nn.BatchNorm3d(out_planes, eps=1e-3, momentum=0.001, affine=True)
        self.relu_s = nn.ReLU()
 
        self.conv_t = nn.Conv3d(out_planes, out_planes, kernel_size=(kernel_size,1,1), stride=(stride,1,1), padding=(padding,0,0), bias=False)
        self.bn_t = nn.BatchNorm3d(out_planes, eps=1e-3, momentum=0.001, affine=True)
        self.relu_t = nn.ReLU()
 
    def forward(self, x):
        x = self.conv_s(x)
        x = self.bn_s(x)
        x = self.relu_s(x)
 
        x = self.conv_t(x)
        x = self.bn_t(x)
        x = self.relu_t(x)
        return x

In [ ]:
class Mixed_3b(nn.Module):
    def __init__(self):
        super(Mixed_3b, self).__init__()
 
        self.branch0 = nn.Sequential(
            BasicConv3d(192, 64, kernel_size=1, stride=1),                        #Branch 1x1x1
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(192, 96, kernel_size=1, stride=1),                        #Branch 1x1x1
            SepConv3d(96, 128, kernel_size=3, stride=1, padding=1),               #Conv3Dx3x3->Conv3D 3x1x1
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(192, 16, kernel_size=1, stride=1),                        #Branch 1x1x1
            SepConv3d(16, 32, kernel_size=3, stride=1, padding=1),                #Conv3Dx3x3->Conv3D 3x1x1
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3,3,3), stride=1, padding=1),               #Branch MaxPool 3x3x3
            BasicConv3d(192, 32, kernel_size=1, stride=1),                        #Conv3D1x1x1
        )
 
    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)                                      #Concatenazione
 
        return out
 

In [ ]:
class Mixed_3c(nn.Module):
    def __init__(self):
        super(Mixed_3c, self).__init__()
        self.branch0 = nn.Sequential(
            BasicConv3d(256, 128, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(256, 128, kernel_size=1, stride=1),
            SepConv3d(128, 192, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(256, 32, kernel_size=1, stride=1),
            SepConv3d(32, 96, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3,3,3), stride=1, padding=1),
            BasicConv3d(256, 64, kernel_size=1, stride=1),
        )
 
    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out
 

In [ ]:
class Mixed_4b(nn.Module):
    def __init__(self):
        super(Mixed_4b, self).__init__()
 
        self.branch0 = nn.Sequential(
            BasicConv3d(480, 192, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(480, 96, kernel_size=1, stride=1),
            SepConv3d(96, 208, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(480, 16, kernel_size=1, stride=1),
            SepConv3d(16, 48, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3,3,3), stride=1, padding=1),
            BasicConv3d(480, 64, kernel_size=1, stride=1),
        )
 
    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out
 

In [ ]:
class Mixed_4c(nn.Module):
    def __init__(self):
        super(Mixed_4c, self).__init__()
 
        self.branch0 = nn.Sequential(
            BasicConv3d(512, 160, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(512, 112, kernel_size=1, stride=1),
            SepConv3d(112, 224, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(512, 24, kernel_size=1, stride=1),
            SepConv3d(24, 64, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3,3,3), stride=1, padding=1),
            BasicConv3d(512, 64, kernel_size=1, stride=1),
        )
 
    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out

In [ ]:
class Mixed_4d(nn.Module):
    def __init__(self):
        super(Mixed_4d, self).__init__()
 
        self.branch0 = nn.Sequential(
            BasicConv3d(512, 128, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(512, 128, kernel_size=1, stride=1),
            SepConv3d(128, 256, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(512, 24, kernel_size=1, stride=1),
            SepConv3d(24, 64, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3,3,3), stride=1, padding=1),
            BasicConv3d(512, 64, kernel_size=1, stride=1),
        )
 
    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out
 

In [ ]:
class Mixed_4e(nn.Module):
    def __init__(self):
        super(Mixed_4e, self).__init__()
 
        self.branch0 = nn.Sequential(
            BasicConv3d(512, 112, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(512, 144, kernel_size=1, stride=1),
            SepConv3d(144, 288, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(512, 32, kernel_size=1, stride=1),
            SepConv3d(32, 64, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3,3,3), stride=1, padding=1),
            BasicConv3d(512, 64, kernel_size=1, stride=1),
        )
 
    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out
 

In [ ]:
class Mixed_4f(nn.Module):
    def __init__(self):
        super(Mixed_4f, self).__init__()
 
        self.branch0 = nn.Sequential(
            BasicConv3d(528, 256, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(528, 160, kernel_size=1, stride=1),
            SepConv3d(160, 320, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(528, 32, kernel_size=1, stride=1),
            SepConv3d(32, 128, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3,3,3), stride=1, padding=1),
            BasicConv3d(528, 128, kernel_size=1, stride=1),
        )
 
    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out

In [ ]:
class Mixed_5b(nn.Module):
    def __init__(self):
        super(Mixed_5b, self).__init__()
 
        self.branch0 = nn.Sequential(
            BasicConv3d(832, 256, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(832, 160, kernel_size=1, stride=1),
            SepConv3d(160, 320, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(832, 32, kernel_size=1, stride=1),
            SepConv3d(32, 128, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3,3,3), stride=1, padding=1),
            BasicConv3d(832, 128, kernel_size=1, stride=1),
        )
 
    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out
 

In [ ]:
class Mixed_5c(nn.Module):
    def __init__(self):
        super(Mixed_5c, self).__init__()
 
        self.branch0 = nn.Sequential(
            BasicConv3d(832, 384, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(832, 192, kernel_size=1, stride=1),
            SepConv3d(192, 384, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(832, 48, kernel_size=1, stride=1),
            SepConv3d(48, 128, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3,3,3), stride=1, padding=1),
            BasicConv3d(832, 128, kernel_size=1, stride=1),
        )
 
    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out

In [ ]:
from torchsummary import summary
summary(S3D(10).cuda(), (3,16, 176, 128))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 64, 16, 88, 64]           9,408
       BatchNorm3d-2       [-1, 64, 16, 88, 64]             128
              ReLU-3       [-1, 64, 16, 88, 64]               0
            Conv3d-4        [-1, 64, 8, 88, 64]          28,672
       BatchNorm3d-5        [-1, 64, 8, 88, 64]             128
              ReLU-6        [-1, 64, 8, 88, 64]               0
         SepConv3d-7        [-1, 64, 8, 88, 64]               0
         MaxPool3d-8        [-1, 64, 8, 44, 32]               0
            Conv3d-9        [-1, 64, 8, 44, 32]           4,096
      BatchNorm3d-10        [-1, 64, 8, 44, 32]             128
             ReLU-11        [-1, 64, 8, 44, 32]               0
      BasicConv3d-12        [-1, 64, 8, 44, 32]               0
           Conv3d-13       [-1, 192, 8, 44, 32]         110,592
      BatchNorm3d-14       [-1, 192, 8,

In [ ]:
from torchvision import transforms
class MyDataLoaderT(Dataset):
  def __init__(self,X,y,root_dir,transform=None):
    self.X=X
    self.y=y
    self.root_dir=root_dir
    self.transform = transforms.Compose([
                                         transforms.ToPILImage(),
                                         transforms.RandomRotation(degrees=25),
                                         transforms.ColorJitter(),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor()
                                         ])
    self.ok=[]
    self.okLab=[]
    return

  def __len__(self):
    return len(self.X)

  def getOk(self):
    return self.ok

  def getOkLab(self):
    return self.okLab
  
  def __getitem__(self,index):
    fname=self.root_dir+self.X[index]+".mp4"
    self.ok.append(self.X[index])
    self.okLab.append(self.y[index])
    video=cv2.VideoCapture(fname)
    img=[]
    r,g,b = [],[],[]
    
    while True:
      # Capture frame-by-frame
      ret , frame=video.read()
      if ret:
        frame= cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        transformer1=torchvision.transforms.ToTensor()
        frame=transformer1(Image.fromarray(frame))
        r1 = frame[0]
        g1 = frame[1]
        b1 = frame[2]
        r.append(r1.unsqueeze(0))
        g.append(g1.unsqueeze(0))
        b.append(b1.unsqueeze(0))
        r1=self.transform(r1)
        g1=self.transform(g1)
        b1=self.transform(b1)
        r.append(r1)
        g.append(g1)
        b.append(b1)
      else:
        break
    r=torch.cat(r)
    g=torch.cat(g)
    b=torch.cat(b)
    img=torch.stack((r,g,b),dim=0)
    return img, self.y[index]
    

In [ ]:
class MyDataLoader(Dataset):
  def __init__(self,X,y,root_dir,transform=None):
    self.X=X
    self.y=y
    self.root_dir=root_dir
    self.transform = transforms.ToTensor()
    self.ok=[]
    self.okLab=[]
    return

  def __len__(self):
    return len(self.X)

  def getOk(self):
    return self.ok

  def getOkLab(self):
    return self.okLab
  
  def __getitem__(self,index):
    fname=self.root_dir+self.X[index]+".mp4"
    self.ok.append(self.X[index])
    self.okLab.append(self.y[index])
    video=cv2.VideoCapture(fname)
    img=[]
    r,g,b = [],[],[]
    
    while True:
      # Capture frame-by-frame
      ret , frame=video.read()
      if ret:
        frame= cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        transformer1=torchvision.transforms.ToTensor()
        frame=transformer1(Image.fromarray(frame))
        r1 = frame[0]
        g1 = frame[1]
        b1 = frame[2]
        r.append(r1.unsqueeze(0))
        g.append(g1.unsqueeze(0))
        b.append(b1.unsqueeze(0))
      else:
        break
    r=torch.cat(r)
    g=torch.cat(g)
    b=torch.cat(b)
    img=torch.stack((r,g,b),dim=0)
    return img, self.y[index]

In [ ]:
def train(epoch,steps,print_every,trainLoader,s):#
    train_loss=[]
    running_loss = 0
 
    for batch_idx, (inputs, labels) in enumerate(trainLoader):  
        inputs, labels= inputs.to(device), labels.to(device)
        logps = model(inputs)
        loss = criterion(logps, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if (batch_idx+1) % print_every == 0:
            s.append('Epoch [{}/{}], Step [{}/{}], Loss: {:.6f}' 
                   .format(epoch, epochs, batch_idx+1, steps, running_loss/(batch_idx*batch_size) ))
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.6f}' 
                   .format(epoch, epochs, batch_idx+1, steps, running_loss/(batch_idx*batch_size)))
 
        train_loss.append(loss.item())
    return np.mean(train_loss)/len(train_loader)

In [ ]:
def val(model,valLoader,s,ret):
    val_loss = 0
    tp = 0
    
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(valLoader):
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model(inputs)
            ret+=logps
            _, predicted = torch.max(logps, 1)
            tp += (predicted == labels).sum().item()
            loss = criterion(logps,labels)
            val_loss += loss.item()
        val_loss /= len(valLoader.dataset)
        
        s.append('\nVal set: Avg. loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, tp, len(valLoader.dataset),
        100. * tp / len(valLoader.dataset)))
        print('\nVal set: Avg. loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, tp, len(valLoader.dataset),
        100. * tp / len(valLoader.dataset)))
        
    return val_loss,tp / len(valLoader.dataset)

In [ ]:
def test(model,testLoader,criterion):
    test_loss = 0
    accuracy = 0
    test_losses=[]
    predictions=[]
    tp=[]
    
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(testLoader):
            inputs, labels= inputs.to(device), labels.to(device)   
            tp+=labels
            logps = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions+=predicted
            accuracy += (predicted == labels).sum().item()
            loss = criterion(logps,labels)
            test_loss += loss.item()
            
        test_loss /= len(testLoader.dataset)
        test_losses.append(test_loss)
        
    return np.mean(test_losses),accuracy_score(np.array(tp).astype(int),np.array(predictions).astype(int))

In [ ]:
def model():
  modello=S3D(10)
  modello.cuda()
  modello.to(device)
  return modello


def checkP(model,epoch,optimizer,criterion,train_loss,val_loss,val_accs,strings,out,name):
  torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'criterion':criterion,
            'train_loss':train_loss,
            'val_loss':val_loss,
            'val_accs':val_accs,
            'strings':strings,
            'out':out
            }, path+"restore-"+name+".model")
  
def loadP(file):
  modello=model()
  checkpoint = torch.load(PATH+"restore-"+file+".model")
  optimizer = torch.optim.Adam(modello.parameters())
  modello.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch = checkpoint['epoch']
  criterion = checkpoint['criterion']
  train_loss = checkpoint['train_loss']
  val_loss = checkpoint['val_loss']
  val_accs = checkpoint['val_accs']
  strings=checkpoint['strings']
  out=checkpoint['out']
  return modello,epoch,optimizer,criterion,train_loss,val_loss,val_accs,strings,out

In [ ]:
def testPrediction(name,size):
    batch_size=size
    model,startEpoch,optimizer,criterion,train_loss,val_loss,val_accs,strings,out=loadP(name)#
    testLoader=DataLoader(dataset=MyDataLoader(X_test,y_test,DATASET_FILES,'test'),batch_size=batch_size)
    predictions=[]
    trueL=[]
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(testLoader):
            inputs, labels= inputs.to(device), labels.to(device)
            logps = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions+=[torch.Tensor.cpu(e).detach() for e in predicted]
            truelabels+=[torch.Tensor.cpu(e).detach() for e in labels]
            
    return np.array(predictions),np.array(trueL)

In [ ]:
def statVal(model,test_loader,criterion):#
    print("Accuratezza sul validation",round(val_accs[-1],2))
    test_loss,test_accs=test(model,test_loader,criterion)
    print("La loss sul test vale ",round(test_loss,3))
    print("L'accuratezza del test vale", round(test_accs*100,2),"%")

def plotParz(name,size):#
    batch_size=size
    model,startEpoch,optimizer,criterion,train_losses,val_losses,val_accs,strings,out=loadP(name)
    testLoader=DataLoader(dataset=MyDataLoader(X_test,y_test,DATASET_FILES,'test'),batch_size=batch_size)
    plt.plot(range(len(train_losses)),np.array(train_losses)*int(len(x_train)/batch_size)/batch_size, label='Training loss')
    plt.plot(range(len(val_losses)),val_losses, label='Validation loss')
    plt.plot(range(len(val_accs)),val_accs, label='Accuracy Validation')
    plt.grid()
    plt.legend(frameon=False)
    plt.show();
    return model,test_loader,criterion

def plotCompl(train_losses,val_losses,val_accs):
    plt.plot(range(len(train_losses)),train_losses, label='Training loss')
    plt.plot(range(len(val_losses)),val_losses, label='Validation loss')
    plt.plot(range(len(val_accs)),val_accs, label='Accuracy Validation')
    plt.legend(frameon=False)
    plt.show();  

In [ ]:
def start(learning_rate):
    modello,startEpoch,criterion=model(),1,nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(modello.parameters(), lr=learning_rate)
    return modello,startEpoch,optimizer,criterion

def restart(fname):
    model,startEpoch,optimizer,criterion,train_loss,val_loss,val_accs,s,output=loadP(fname)
    print("Ok ripristino!")
    return model,startEpoch,optimizer,criterion,train_loss,val_loss,val_accs,s,output

def salva(model,epoch,optimizer,criterion,train_loss,val_loss,val_accs,s,output,FNAME):
     checkP(model,epoch,optimizer,criterion,train_loss,val_loss,val_accs,s,output,FNAME)
     print("salvato!")
    
        

In [ ]:
RESTORE=False
FNAME="zero"
epochs=11
batch_size=32
learning_rate=0.0001
print_every=100
s=[]
output=[]
train_loss = []
val_loss = []
val_accs=[]


if not RESTORE:
    model,startEpoch,optimizer,criterion=start(learning_rate)
else:
    model,startEpoch,optimizer,criterion,train_loss,val_loss,val_accs,s,output=restart(FNAME)

trainLoader= DataLoader(dataset=MyDataLoaderT(X_train,y_train,DATASET_FILES),batch_size=batch_size)
valLoader=DataLoader(dataset=MyDataLoader(X_val,y_val,DATASET_FILES),batch_size=batch_size)
model=model.to(device)
epoch=0

try:
    x=[]
    val(model,valLoader,s,x)
    step=len(trainLoader)
    output.append(x)
    for epoch in range(startEpoch,epochs):
            outx=[]
            sx=[]
            train_loss1=train(epoch,step,print_every,trainLoader,sx)
            val_loss1,val_accs1=val(model,valLoader,sx,outx)
            train_loss.append(train_loss1)
            val_loss.append(val_loss1)
            val_accs.append(val_accs1) 
            output.append(outx)
            s+=sx
    
except KeyboardInterrupt: 
    salva(model,epoch,optimizer,criterion,train_loss,val_loss,val_accs,s,output,FNAME)

salva(model,epoch,optimizer,criterion,train_loss,val_loss,val_accs,s,output,FNAME)
plotCompl(train_loss,val_loss,val_accs)
  


Val set: Avg. loss: 0.076873, Accuracy: 65/260 (25%)



RuntimeError: ignored